In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/face3d/face3d/mesh/cython

/content/gdrive/MyDrive/face3d/face3d/mesh/cython


In [3]:
!python setup.py build_ext --inplace


running build_ext
skipping 'mesh_core_cython.cpp' Cython extension (up-to-date)


In [4]:
!python setup.py install

running install
running build
running build_ext
skipping 'mesh_core_cython.cpp' Cython extension (up-to-date)
running install_lib
copying build/lib.linux-x86_64-3.8/mesh_core_cython.cpython-38-x86_64-linux-gnu.so -> /usr/local/lib/python3.8/dist-packages
running install_egg_info
Writing /usr/local/lib/python3.8/dist-packages/mesh_core_cython-0.0.0.egg-info


In [9]:
import sys
sys.path.append('/content/gdrive/MyDrive/face3d/')
import face3d
from face3d import mesh
from face3d.morphable_model import MorphabelModel

In [20]:
import cv2
import dlib
import numpy as np
from skimage import io
import os

In [47]:
im = cv2.imread('/content/gdrive/MyDrive/face3d/examples/Data/wu.jpg')
im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
h, w, c = im.shape
# 人脸检测
detector = dlib.get_frontal_face_detector()
# 68关键点检测
predictor = dlib.shape_predictor("/content/gdrive/MyDrive/face3d/examples/Data/shape_predictor_68_face_landmarks.dat")

In [56]:
# 人脸检测框
rects = detector(im, 1)

for k, d in enumerate(rects):
    print("dets{}".format(d))
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
 
    # 使用predictor进行人脸关键点识别 shape为返回的结果
    shape = predictor(im, d)
    # 获取第一个和第二个点的坐标（相对于图片而不是框出来的人脸）
    print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))

    for index, pt in enumerate(shape.parts()):
        print('Part {}: {}'.format(index, pt))
        pt_pos = (pt.x, pt.y)
        cv2.circle(im, pt_pos, 1, (255, 0, 0), 2)
        # 利用cv2.putText输出1-68
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(im, str(index + 1), pt_pos, font, 0.3, (0, 0, 255), 1, cv2.LINE_AA)


# 关键点(68)
shape = predictor(im, rects[0])
rects = [(rects[0].tl_corner().x, rects[0].tl_corner().y), 
         (rects[0].br_corner().x, rects[0].br_corner().y)]
landmarks = np.zeros((68, 2))
# 画出关键点
for i, p in enumerate(shape.parts()):
    landmarks[i] = [p.x, p.y]

dets[(277, 63) (598, 384)]
Detection 0: Left: 277 Top: 63 Right: 598 Bottom: 384
Part 0: (281, 136), Part 1: (286, 181) ...
Part 0: (281, 136)
Part 1: (286, 181)
Part 2: (295, 225)
Part 3: (304, 268)
Part 4: (317, 309)
Part 5: (338, 343)
Part 6: (368, 373)
Part 7: (403, 396)
Part 8: (444, 403)
Part 9: (485, 393)
Part 10: (521, 369)
Part 11: (554, 340)
Part 12: (579, 304)
Part 13: (592, 261)
Part 14: (599, 216)
Part 15: (606, 169)
Part 16: (610, 122)
Part 17: (297, 108)
Part 18: (316, 90)
Part 19: (345, 87)
Part 20: (373, 95)
Part 21: (400, 107)
Part 22: (449, 106)
Part 23: (477, 93)
Part 24: (509, 84)
Part 25: (541, 85)
Part 26: (567, 102)
Part 27: (424, 139)
Part 28: (424, 170)
Part 29: (423, 200)
Part 30: (423, 232)
Part 31: (395, 250)
Part 32: (410, 255)
Part 33: (427, 258)
Part 34: (444, 254)
Part 35: (462, 248)
Part 36: (327, 142)
Part 37: (345, 132)
Part 38: (366, 133)
Part 39: (387, 147)
Part 40: (365, 152)
Part 41: (344, 152)
Part 42: (472, 144)
Part 43: (490, 129)
Part 44: (51

In [57]:
def from_image(vertices, h, w):
    ''' change vertices to image coord system
    3d system: XYZ, center(0, 0, 0)
    2d image: x(u), y(v). center(w/2, h/2), flip y-axis.
    Args:
        vertices: [nver, 3]
        h: height of the rendering
        w : width of the rendering
    Returns:
        projected_vertices: [nver, 3]
    '''
    image_vertices = vertices.copy()

    # move to center of image
    image_vertices[:,0] = image_vertices[:,0] - w/2
    # flip vertices along y-axis.
    image_vertices[:,1] = -(- h + image_vertices[:,1] + 1)
    image_vertices[:, 1] = image_vertices[:, 1] - h / 2
    return image_vertices

In [59]:
# load bfm模型
bfm = MorphabelModel('/content/gdrive/MyDrive/face3d/examples/Data/BFM/Out/BFM.mat')
print('init bfm model success')
# 坐标系变换
x = from_image(landmarks, h, w)
X_ind = bfm.kpt_ind

init bfm model success


In [60]:
# fit 传入关键点2d坐标及对应模型中顶点的索引
fitted_sp, fitted_ep, fitted_s, fitted_angles, fitted_t = bfm.fit(x, X_ind, max_iter = 200)
# texture的系数直接随机
colors = bfm.generate_colors(np.random.rand(bfm.n_tex_para, 1))
colors = np.minimum(np.maximum(colors, 0), 1)

In [61]:
fitted_vertices = bfm.generate_vertices(fitted_sp, fitted_ep)

In [62]:
# 包含平移, 旋转, 缩放
transformed_vertices = bfm.transform(fitted_vertices, fitted_s, fitted_angles, fitted_t)

In [63]:
image_vertices = mesh.transform.to_image(transformed_vertices, h, w)

In [64]:
fitted_image = mesh.render.render_colors(image_vertices, bfm.triangles, colors, h, w)

In [65]:
save_folder = '/content/gdrive/MyDrive/face3d/results'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

io.imsave('{}/generated.jpg'.format(save_folder), im)
io.imsave('{}/fitted.jpg'.format(save_folder), fitted_image)